In [2]:
#import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import os

proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset7.csv')
df = pd.read_csv(train_path)

labels=df['Churn']
x = df.drop(columns=['Churn'],axis = 'columns')
y=np.ravel(labels)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

oversample = SMOTE()
x_train, y_train = oversample.fit_resample(x_train, y_train)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=14, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,
      decay_steps=(x_train.shape[0]/32)*50,
      decay_rate=1,
      staircase=False)
def get_optimizer():
    return tf.keras.optimizers.Adam(lr_schedule)

nn_model.compile(loss = "binary_crossentropy", 
                  optimizer = get_optimizer(), 
                  metrics=['accuracy'])

callback =tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=70,restore_best_weights=True)
history = nn_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])

y_pred = nn_model.predict(x_test)

print ('accuracy : {0:.2f}'.format(accuracy_score(y_test,y_pred.round())))
print ('precision  : {0:.2f}'.format(precision_score(y_test,y_pred.round())))
print ('recall  : {0:.2f}'.format(recall_score(y_test,y_pred.round())))
print ('f1 score  : {0:.2f}'.format(f1_score(y_test,y_pred.round())))
print ('confusion matrix : \n {0}'.format(confusion_matrix(y_test,y_pred.round())))

accuracy : 0.71
precision  : 0.71
recall  : 0.68
f1 score  : 0.69
confusion matrix : 
 [[226  80]
 [ 95 199]]
